In [3]:
import requests as r
from  bs4 import BeautifulSoup
from lib.toolbox import gen_header
import math
import threading
import time
import re
from random import randint

host = 'http://www.sum.com.tw/'
carName_list = ['VOLVO','VW','SUZUKI','SUBARU','PORSCHE','AUDI','BENZ','BMW','LEXUS','FORD','TOYOTA','MAZDA','HONDA','MITSUBISHI','NISSAN']

In [ ]:
def creatTB():
    db = sqlite3.connect("./SUM.db")
    cur = db.cursor()
    cur.execute("drop table SUM")
    cursor.execute('''CREATE TABLE icars
             (source text, url text, title text, brand text, model text, doors text, color text,
             cc text, transmission text, equip text, mileage text, years text, location text, posttime text, price text,
             certificate text)''')
    db.commit()
creatTB()


In [8]:
#format
url = "http://www.sum.com.tw/carinfo.php?id=394792"
res = r.get(url)
soup = BeautifulSoup(res.text,'lxml')
formatList = soup.select('.formatArea > .formatList > li')
print(len(formatList))
for carformat in formatList:
    for span in carformat.find_all('span'):
        span.decompose()
print(formatList[0].text) #
print(formatList[1].text)   #
print(formatList[2].text)
print(formatList[3].text) #
print(formatList[4].text)
print(formatList[5].text)
print(formatList[8].text)

9
自排
汽油車 0
2WD
5門
5




In [13]:
#for idx, tr in enumerate(td):

In [29]:
import time
url = "http://www.sum.com.tw/carinfo.php?id=394792"
res = r.get(url,headers=HEADER)
soup = BeautifulSoup(res.text,'lxml')
infoList = soup.select('.infoList > li')

print(infoList)
query_list = [] 

query_dict['brand'] = infoList[0].text.strip()
query_dict['years'] = infoList[1].text
query_dict['color'] = infoList[2].text
query_dict['cc'] = infoList[3].text

#time  tm
# po_time = infoList[4].text
# print(po_time)
# poTimeArray = time.strptime(po_time, "%Y.%m.%d")
# tm = int(time.mktime(poTimeArray))
# query_dict['time'] = tm


query_dict["price"] = soup.select_one('.price > span > strong').text
query_dict
a=soup.select('.lineBottom')[3].text
a

[<li><strong>廠牌車款</strong>SUZUKI - SWIFT </li>, <li><strong>出廠年份</strong>2011</li>, <li><strong>外觀顏色</strong>紅色系</li>, <li><strong>排  氣  量</strong>1400c.c</li>, <li><strong>展  售  日</strong>2017.03.10</li>]


'雲林縣虎尾鎮竹圍68號'

In [19]:
def gen_headers():
    referers = ['tw.yahoo.com', 'www.google.com', 'http://www.msn.com/zh-tw/', 'http://www.pchome.com.tw/']
    user_agents = [
        'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
        'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)']
    headers = {'User-Agent': user_agents[randint(0, len(user_agents) - 1)],
               'Remferer': referers[randint(0, len(referers) - 1)]}
    return headers

In [22]:
def main():
    for carName in carName_list:
        with open('./SUM/{}.csv'.format(carName), 'r') as fr:
            url = fr.read().strip().split()
            get_content(url, carName)

In [33]:
def get_content(url, carName):
    
    headers = gen_headers()
    res = r.get(url,headers=HEADER)
    soup = BeautifulSoup(res.text,'lxml')
    
    infoList = soup.select('.infoList > li')
    for idx in infoList:
        for strong in idx.find_all('strong'):
            strong.decompose()
    
    formatList = soup.select('.formatArea > .formatList > li')
#     print(len(formatList))
    for carformat in formatList:
        for span in carformat.find_all('span'):
            span.decompose()
            
    outfitList = soup.select('.formatArea > .outfitList > .have')
    contents_dict = {}
    contents = []
    
    #brand
    brand = carName
    contents.append('廠牌：' + brand)
    
    # price
    price = soup.select_one('.price > span > strong').text
    contents.append('價格：' + price)

    # posTime
    po_time = infoList[4].text
    poTimeArray = time.strptime(po_time, "%Y.%m.%d")
    tm = time.mktime(poTimeArray)
    contents.append('時間：' + str(tm))

    # title
    title_table = soup.select_one('.gradientGray_bar > h1').text
    contents.append('標題：' + title_table)
    
    # Eqip
    outfits = ""
    for outfit in outfitList:
        outfits += outfit.text.strip()
        outfits += ","
    contents.append('配備：' + outfits)

    content_dict = {k.split('：')[0]: k.split('：')[1] for k in contents if len(k) > 0}
    
    content_dict['網站'] = 'SUM'
    content_dict['連結'] = url
    #transmission 
    content_dict['排擋方式'] = formatList[0].text
    #doors
    content_dict['車門'] = formatList[3].text
    #years
    content_dict['年份'] = infoList[1].text
    #color
    content_dict['色系'] = infoList[2].text
    #location
    content_dict['所在地'] = soup.select('.lineBottom')[3].text
    #model
    content_dict['型號'] = infoList[0].text.strip()
    
    return content_dict

get_content("http://www.sum.com.tw/carinfo.php?id=394792","SUZUKI")

{'價格': '36.8萬',
 '型號': 'SUZUKI - SWIFT',
 '年份': '2011',
 '廠牌': 'SUZUKI',
 '所在地': '雲林縣虎尾鎮竹圍68號',
 '排擋方式': '自排',
 '時間': '1489075200.0',
 '標題': 'SUZUKI SWIFT  2011 SWIFT 頂級I-KEY',
 '網站': 'SUM',
 '色系': '紅色系',
 '車門': '5門',
 '連結': 'http://www.sum.com.tw/carinfo.php?id=394792',
 '配備': 'CD,ABS,TCS,HID頭燈,皮椅,定速,防盜,鋁圈,霧燈,備胎,一手車,電動窗,中控鎖,安全氣囊,恆溫空調,音響主機,倒車雷達,晶片系統,電動後視鏡,核桃木飾板,車前距雷達,行車資訊顯示器,方向盤音響控制,免KEY,'}

In [38]:
def add_to_sqlite(content_dict):
    conn = sqlite3.connect('./SUM.db')
    cursor = conn.cursor()

    source = content_dict['網站']   #
    url = content_dict['連結']      #
    title = content_dict['標題']    #
    brand = content_dict['廠牌']    #
    model = content_dict['型號']    #
    doors = content_dict['車門']    #
    color = content_dict['色系']    #
    cc = content_dict['排氣量']     #
    transmission = content_dict['排擋方式']   #
    equip = content_dict['配備']            #
    mileage = content_dict['行駛里程']      #
    years = content_dict['年份']           #
    location = content_dict['所在地']     #
    posttime = content_dict['時間']      #
    price = content_dict['價格']        #
    certificate = ''

    cursor.execute('INSERT INTO icars VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
                   (source, url, title, brand, model, doors, color, cc, transmission, equip,
                    mileage, years, location, posttime, price, certificate))

    conn.commit()

In [39]:
def check_duplicates(content_dict):
    conn = sqlite3.connect('/home/ubuntu/python/III/Python/final_project/Crawler/icars.db')
    cursor = conn.cursor()

    title = content_dict['標題']
    color = content_dict['色系']
    cc = content_dict['排氣量']
    equip = content_dict['配備']
    location = content_dict['所在地']

    num = list(
        cursor.execute('SELECT * FROM icars WHERE title = ? AND cc = ? AND color = ?  AND equip = ? AND location = ?',
                       (title, cc, color, equip, location)))
    if num != 0:
        return False
    else:
        return True

In [ ]:
conn = sqlite3.connect('./Mobile01.db')
cursor = conn.cursor()

source = content_dict['網站']
url = content_dict['連結']
title = content_dict['標題']
brand = content_dict['廠牌']
model = content_dict['型號']
doors = content_dict['車門']
color = content_dict['色系']
cc = content_dict['排氣量']
transmission = content_dict['排擋方式']
equip = content_dict['配備']
mileage = content_dict['網站']
years = content_dict['行駛里程']
location = content_dict['所在地']
posttime = content_dict['時間']
price = content_dict['價格']
certificate = ''


try:
    cursor.execute('INSERT INTO icars VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
                   (source, url, title, brand, model, doors, color, cc, transmission, equip,
                    mileage, years, location, posttime, price, certificate))
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback() 